In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from types import SimpleNamespace

In [ ]:
# Checkpoint path
# Always safe specific config with a run! Important especially if seed is changed because then we have other splits!

ckpt_path = "/Users/niklaskiermeyer/Desktop/Codespace/DruxAI/results/training/ckpt_sgd_baseline.pt"

checkpoint = torch.load(ckpt_path)
cfg = checkpoint["config"]

In [ ]:
config = {
    "metric": {"name": "r2_val", "goal": "maximize"},
    "resume": False,
    "patience": 5,
    "epochs": 10,
    "optimizer": "sgd",
    "scheduler": "exponential",
    "loss": "huber",
    "batch_size": 64,
    "learning_rate": 0.1,
    "output_features": 10,
    "hidden_dims_drug_nn": [512],
    "hidden_dims_gene_expression_nn": [512],
    "dropout_drug_nn": 0.2,
    "dropout_gene_expression_nn": 0.2,
}

config = SimpleNamespace(**config)